# Estudio de las estadísticas de un robot de limpieza reactivo

## Arturo Cristián Díaz López - A01709522

Dado:

Habitación de MxN espacios.
Número de agentes.
Porcentaje de celdas inicialmente sucias.
Tiempo máximo de ejecución.
Realiza la siguiente simulación:

Inicializa las celdas sucias (ubicaciones aleatorias).
Todos los agentes empiezan en la celda [1,1].
En cada paso de tiempo:
Si la celda está sucia, entonces aspira.
Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).
Se ejecuta el tiempo máximo establecido.
Para un espacio de 100x100, considera los siguientes escenarios:

Escenario 1: 1 agente, 90% de celdas sucias.
Escenario 2. 2 agentes, 90% de celdas sucias.
Deberás resolver las siguientes preguntas:

¿Cuántos pasos de simulación toma limpiar todo el espacio?
¿Qué porcentaje de celdas sucias queda con los siguientes pasos de simulación: 100, 1000, 10000?
A continuación, determina cuál es la cantidad óptima de aspiradoras que debe de tener para realizar la limpieza en el menor tiempo posible. Considera que tenemos un máximo de 10 aspiradoras disponibles.

Desarrollar un informe con lo observado.

In [73]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation

import numpy as np
import pandas as pd

import time
import random

In [74]:
class CleaningAgent(Agent):
    def move(self):
        empty_cell = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        empty_cell = [i for i in empty_cell if self.model.grid.is_cell_empty(i)]

        if empty_cell:
            new_position = random.choice(empty_cell)
            self.model.grid.move_agent(self, new_position)

    def step(self):
        x, y = self.pos

        if self.model.is_clean(x, y):
            self.move()

        else:
            self.model.clean(x, y)
            self.move()

In [75]:
class CleaningModel(Model):
    def __init__(self, width, height, num_agents, dirty_cells):
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)

        self.dirty = np.zeros((width, height))
        self.dirty_cells = int(width * height * dirty_cells)
        self.clean_cells = 0

        self.current_id = 0
        self.steps = 0

        for _ in range(num_agents):
            agent = CleaningAgent(self.next_id(), self)
            self.schedule.add(agent)
            x, y = self.random_empty_cell()
            self.grid.place_agent(agent, (x, y))

        for _ in range(self.dirty_cells):
            x, y = self.random_empty_cell()
            self.dirty[x][y] = 1

    def step(self):
        self.schedule.step()
        self.steps += 1

    def clean(self, x, y):
        self.dirty[x][y] = 0
        self.clean_cells += 1

    def is_clean(self, x, y):
        return self.dirty[x][y] == 0

    def random_empty_cell(self):
        x, y = self.random_pos()
        while not self.grid.is_cell_empty((x, y)) or self.dirty[x][y] == 1:
            x, y = self.random_pos()
        return x, y

    def random_pos(self):
        return random.randrange(self.grid.width), random.randrange(self.grid.height)

    def get_dirty_percentage(self):
        return (self.dirty_cells - self.clean_cells) / self.dirty_cells * 100

    def next_id(self):
        self.current_id += 1
        return self.current_id

In [76]:
WIDTH = 100
HEIGHT = 100
DIRT_PERCENTAGE_1 = 0.9
MAX_STEPS = 10000

In [77]:
NUM_AGENTS_1 = 1

start_time_1 = time.time()
model_1 = CleaningModel(WIDTH, HEIGHT, NUM_AGENTS_1, DIRT_PERCENTAGE_1)

initial_dirty_percentage_1 = model_1.get_dirty_percentage()

for i in range(MAX_STEPS):
    if model_1.get_dirty_percentage() == 0:
        break
    model_1.step()

final_dirty_percentage_1 = model_1.get_dirty_percentage()
end_time_1 = time.time() - start_time_1

print("E S C E N A R I O  1 ")
print("\nInitial dirty percentage: ", initial_dirty_percentage_1)
print("Final dirty percentage: ", final_dirty_percentage_1)
print("Time taken: ", end_time_1)
print("Steps taken: ", model_1.steps)

E S C E N A R I O  1 

Initial dirty percentage:  100.0
Final dirty percentage:  67.58888888888889
Time taken:  0.10799837112426758
Steps taken:  10000


In [78]:
NUM_AGENTS_2 = 2

start_time_2 = time.time()
model_2 = CleaningModel(WIDTH, HEIGHT, NUM_AGENTS_2, DIRT_PERCENTAGE_1)

initial_dirty_percentage_2 = model_2.get_dirty_percentage()

for i in range(MAX_STEPS):
    if model_2.get_dirty_percentage() == 0:
        break
    model_2.step()

final_dirty_percentage_2 = model_2.get_dirty_percentage()
end_time_2 = time.time() - start_time_2

print("E S C E N A R I O  2 ")
print("\nInitial dirty percentage: ", initial_dirty_percentage_2)
print("Final dirty percentage: ", final_dirty_percentage_2)
print("Time taken: ", end_time_2)
print("Steps taken: ", model_2.steps)

E S C E N A R I O  2 

Initial dirty percentage:  100.0
Final dirty percentage:  56.97777777777778
Time taken:  0.17123818397521973
Steps taken:  10000
